# 🔬 ISIC 2018 Deri Lezyonu Segmentasyonu ve Öznitelik Çıkarımı

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KULLANICI_ADI/isic_segmentation_project/blob/main/notebooks/ISIC_Segmentation_Colab.ipynb)

**Proje:** Kapsamlı görüntü işleme pipeline'ı ile otomatik segmentasyon

**Yazar:** Mustafa Engin Dalgıç

**Veri Seti:** ISIC 2018 (2,239 görüntü, 9 sınıf)

---

## 📋 Bu Notebook'ta:

1. 🔧 Kurulum ve bağımlılıklar
2. 📂 Veri seti yükleme (Google Drive / Kaggle)
3. 🎯 Pipeline çalıştırma (5 aşama)
4. 📊 Sonuçları görselleştirme
5. 💾 Çıktıları kaydetme

**⏱️ Tahmini Süre:** ~15-30 dakika (2,239 görüntü)

**💾 Gerekli Alan:** ~500 MB

## 1️⃣ Kurulum ve Bağımlılıklar

Gerekli Python kütüphanelerini yükleyelim:

In [32]:
# Colab ortamında mı kontrol et
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colab ortamında çalışıyorsunuz")
except:
    IN_COLAB = False
    print("ℹ️  Lokal ortamda çalışıyorsunuz")

# Repository'yi klonla
if IN_COLAB:
    # Eğer varsa eski klonu sil
    import os
    if os.path.exists('UU_DIP_isic_segmentation_v2'):
        !rm -rf UU_DIP_isic_segmentation_v2

    # Klonla
    !git clone https://github.com/engindalgic86-cpu/UU_DIP_isic_segmentation_v2.git

    # Dizine geç - os.chdir kullan (daha güvenilir)
    import os
    os.chdir('UU_DIP_isic_segmentation_v2')
    print(f"📂 Mevcut dizin: {os.getcwd()}")

# Bağımlılıkları yükle
!pip install -q -r requirements.txt

print("\n✅ Kurulum tamamlandı!")

✅ Google Colab ortamında çalışıyorsunuz
Cloning into 'UU_DIP_isic_segmentation_v2'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 19 (delta 4), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 32.10 KiB | 3.57 MiB/s, done.
Resolving deltas: 100% (4/4), done.
[Errno 2] No such file or directory: 'UU_DIP_isic_segmentation_v2 # ← ÇOK ÖNEMLİ!'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

✅ Kurulum tamamlandı!


## 2️⃣ Kütüphaneleri İçe Aktar

In [28]:
import sys
sys.path.append('src/')

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Colab için matplotlib ayarları
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['figure.dpi'] = 100

print("✅ Kütüphaneler yüklendi!")
print(f"   • OpenCV: {cv2.__version__}")
print(f"   • NumPy: {np.__version__}")
print(f"   • Pandas: {pd.__version__}")

✅ Kütüphaneler yüklendi!
   • OpenCV: 4.12.0
   • NumPy: 2.0.2
   • Pandas: 2.2.2


## 3️⃣ Veri Seti Yükleme

**Seçenek A:** Google Drive'dan yükleme (önerilen)

**Seçenek B:** Kaggle API ile indirme

### Seçenek A: Google Drive'dan Yükleme

In [29]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    # ISIC veri setinin Drive'daki yolu
    # NOT: Yolunuzu kendi Drive yapınıza göre güncelleyin
    DRIVE_ISIC_PATH = '/content/drive/MyDrive/ISIC_2018'

    # Veri yolunu ayarla
    import os
    if os.path.exists(DRIVE_ISIC_PATH):
        DATA_PATH = DRIVE_ISIC_PATH
        print(f"✅ Veri seti bulundu: {DATA_PATH}")
        print(f"   Görüntü sayısı: {len(os.listdir(DATA_PATH))}")
    else:
        print(f"❌ Veri seti bulunamadı: {DRIVE_ISIC_PATH}")
        print("\n⚠️  Lütfen ISIC 2018 veri setini Google Drive'a yükleyin.")
        print("   Veya DRIVE_ISIC_PATH değişkenini güncelleyin.")
        DATA_PATH = None
else:
    DATA_PATH = 'data/ISIC'  # Lokal yol
    print(f"ℹ️  Lokal veri yolu: {DATA_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
❌ Veri seti bulunamadı: /content/drive/MyDrive/ISIC_2018

⚠️  Lütfen ISIC 2018 veri setini Google Drive'a yükleyin.
   Veya DRIVE_ISIC_PATH değişkenini güncelleyin.


### Seçenek B: Kaggle API ile İndirme

In [30]:
# Kaggle API kullanarak indirme (alternatif)
USE_KAGGLE = True  # True yapın eğer Kaggle'dan indirmek isterseniz

if USE_KAGGLE and IN_COLAB:
    # Kaggle token'ınızı yükleyin
    from google.colab import files
    print("📁 Lütfen kaggle.json dosyanızı yükleyin...")
    files.upload()  # kaggle.json dosyasını seçin

    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    # Veri setini indir
    !kaggle datasets download -d nodoubttome/skin-cancer9-classesisic
    !unzip -q skin-cancer9-classesisic.zip -d data/

    DATA_PATH = 'data/'
    print("✅ Veri seti indirildi!")

📁 Lütfen kaggle.json dosyanızı yükleyin...


Saving kaggle.json to kaggle (3).json
Dataset URL: https://www.kaggle.com/datasets/nodoubttome/skin-cancer9-classesisic
License(s): other
skin-cancer9-classesisic.zip: Skipping, found more recently modified local copy (use --force to force download)
replace data/Skin cancer ISIC The International Skin Imaging Collaboration/Test/actinic keratosis/ISIC_0010512.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace data/Skin cancer ISIC The International Skin Imaging Collaboration/Test/actinic keratosis/ISIC_0010889.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace data/Skin cancer ISIC The International Skin Imaging Collaboration/Test/actinic keratosis/ISIC_0024468.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
✅ Veri seti indirildi!


## 4️⃣ Pipeline'ı Yükle ve Çalıştır

Ana segmentasyon pipeline'ını yükleyelim:

In [31]:
# Ana script'i import et
exec(open('src/isic_segmentation_project.py').read())

print("✅ Pipeline yüklendi!")
print("\n📋 Mevcut fonksiyonlar:")
print("   • load_image_dataset()")
print("   • stage1_rgb_to_grayscale()")
print("   • stage2_1_dynamic_crop()")
print("   • stage2_2_contrast_enhancement()")
print("   • stage2_3_noise_reduction()")
print("   • stage3_thresholding_segmentation()")
print("   • stage4_1_morphological_operations()")
print("   • stage4_2_connected_component_labeling()")
print("   • stage5_feature_extraction()")

FileNotFoundError: [Errno 2] No such file or directory: 'src/isic_segmentation_project.py'

## 5️⃣ Veri Setini Yükle

In [ ]:
if DATA_PATH is None:
    print("❌ Veri yolu ayarlanmadı! Lütfen yukarıdaki hücreleri çalıştırın.")
else:
    print(f"📂 Veri seti yükleniyor: {DATA_PATH}")
    df = load_image_dataset(DATA_PATH)

    print(f"\n✅ Veri seti yüklendi!")
    print(f"   Toplam görüntü: {len(df)}")
    print(f"\n📊 Sınıf dağılımı:")
    print(df['class'].value_counts())

## 6️⃣ Pipeline'ı Çalıştır (Demo - 9 Örnek)

Önce 9 örnek üzerinde pipeline'ı test edelim:

In [ ]:
print("🚀 DEMO: 9 örnek üzerinde pipeline çalıştırılıyor...\n")

# Aşama 1
print("=" * 80)
print("AŞAMA 1: RGB → Grayscale")
print("=" * 80)
grayscale_data = stage1_rgb_to_grayscale(df, num_samples=9, save_output=True)

# Aşama 2.1
print("\n" + "=" * 80)
print("AŞAMA 2.1: Dinamik Crop")
print("=" * 80)
cropped_data = stage2_1_dynamic_crop(grayscale_data, num_samples=9, save_output=True)

# Aşama 2.2
print("\n" + "=" * 80)
print("AŞAMA 2.2: Kontrast İyileştirme")
print("=" * 80)
enhanced_data, recommendation = stage2_2_contrast_enhancement(cropped_data, num_samples=9, save_output=True)

# Aşama 2.3
print("\n" + "=" * 80)
print("AŞAMA 2.3: Gürültü Azaltma")
print("=" * 80)
blurred_data, best_kernel = stage2_3_noise_reduction(enhanced_data, num_samples=9, save_output=True)

# Aşama 3
print("\n" + "=" * 80)
print("AŞAMA 3: Thresholding")
print("=" * 80)
segmented_data, best_method = stage3_thresholding_segmentation(blurred_data, num_samples=9, save_output=True)

# Aşama 4.1
print("\n" + "=" * 80)
print("AŞAMA 4.1: Morfolojik Operatörler")
print("=" * 80)
morphed_data, kernel_info = stage4_1_morphological_operations(segmented_data, num_samples=9, save_output=True)

# Aşama 4.2
print("\n" + "=" * 80)
print("AŞAMA 4.2: Connected Component Labeling")
print("=" * 80)
final_data = stage4_2_connected_component_labeling(morphed_data, num_samples=9, save_output=True)

print("\n" + "=" * 80)
print("✅ DEMO TAMAMLANDI!")
print("=" * 80)
print("\n📁 Oluşturulan görselleştirmeler:")
print("   1. 01_rgb_vs_grayscale.png")
print("   2. 02_dynamic_crop.png")
print("   3. 03_contrast_comparison.png")
print("   4. 03_histogram_analysis.png")
print("   5. 04_median_blur_comparison.png")
print("   6. 05_threshold_comparison.png")
print("   7. 06_morphology.png")
print("   8. 07_ccl_final_roi.png")

## 7️⃣ Tüm Veri Setinde Öznitelik Çıkarımı

⚠️ **DİKKAT:** Bu hücre ~15-30 dakika sürebilir (2,239 görüntü)

In [ ]:
# Kullanıcıdan onay al
RUN_FULL_PIPELINE = True  # False yaparak atlayabilirsiniz

if RUN_FULL_PIPELINE:
    print("🚀 TÜM VERİ SETİNDE ÖZNİTELİK ÇIKARIMI BAŞLIYOR...")
    print(f"   Toplam görüntü: {len(df)}")
    print(f"   Tahmini süre: ~{len(df) * 2 / 60:.0f} dakika\n")

    # Aşama 5: Öznitelik çıkarımı
    features_df = stage5_feature_extraction(df, final_data, save_output=True)

    print("\n" + "=" * 80)
    print("🎉 TÜM PROJE TAMAMLANDI!")
    print("=" * 80)
    print(f"\n✅ features.csv oluşturuldu!")
    print(f"   • Satır sayısı: {len(features_df)}")
    print(f"   • Sütun sayısı: {len(features_df.columns)}")
else:
    print("ℹ️  Tam pipeline atlandı. RUN_FULL_PIPELINE = True yaparak çalıştırabilirsiniz.")

## 8️⃣ Sonuçları Görüntüle

In [ ]:
if RUN_FULL_PIPELINE:
    # features.csv'yi yükle
    features_df = pd.read_csv('features.csv')

    print("📊 ÖZNİTELİK TABLOSU ÖZETİ")
    print("=" * 80)
    print(f"\nBoyut: {features_df.shape}")
    print(f"\nİlk 5 satır:")
    display(features_df.head())

    print(f"\nİstatistikler:")
    display(features_df.describe())

## 9️⃣ Görselleştirmeleri Göster

In [ ]:
import os
from IPython.display import Image, display

image_files = [
    '01_rgb_vs_grayscale.png',
    '02_dynamic_crop.png',
    '03_contrast_comparison.png',
    '05_threshold_comparison.png',
    '06_morphology.png',
    '07_ccl_final_roi.png'
]

for img_file in image_files:
    if os.path.exists(img_file):
        print(f"\n📊 {img_file}")
        print("=" * 80)
        display(Image(filename=img_file))
    else:
        print(f"⚠️  {img_file} bulunamadı")

## 🔟 Çıktıları Google Drive'a Kaydet

In [ ]:
if IN_COLAB and RUN_FULL_PIPELINE:
    # Çıktıları Drive'a kopyala
    output_drive_path = '/content/drive/MyDrive/ISIC_Results'

    !mkdir -p {output_drive_path}
    !cp features.csv {output_drive_path}/
    !cp *.png {output_drive_path}/

    print(f"✅ Çıktılar Google Drive'a kaydedildi!")
    print(f"   Konum: {output_drive_path}")
    print(f"\n📁 Kaydedilen dosyalar:")
    !ls -lh {output_drive_path}
else:
    print("ℹ️  Çıktılar mevcut dizinde (Colab'ın geçici depolama alanı)")

## 1️⃣1️⃣ Hızlı Analiz ve Görselleştirme

In [ ]:
if RUN_FULL_PIPELINE:
    # ROI boyut dağılımı
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.hist(features_df['roi_area'], bins=50, edgecolor='black')
    plt.xlabel('ROI Area (pixels)')
    plt.ylabel('Frequency')
    plt.title('ROI Size Distribution')
    plt.grid(alpha=0.3)

    plt.subplot(1, 2, 2)
    class_counts = features_df['class'].value_counts()
    plt.barh(class_counts.index, class_counts.values)
    plt.xlabel('Count')
    plt.title('Class Distribution')
    plt.tight_layout()
    plt.show()

    # Sınıf bazında ROI istatistikleri
    print("\n📊 Sınıf Bazında ROI İstatistikleri:")
    print("=" * 80)
    class_stats = features_df.groupby('class')['roi_area'].agg(['count', 'mean', 'std', 'min', 'max'])
    display(class_stats.sort_values('mean', ascending=False))

## ✅ Özet ve Sonuç

Bu notebook'ta:

- ✅ ISIC 2018 veri seti yüklendi
- ✅ 5 aşamalı segmentasyon pipeline'ı çalıştırıldı
- ✅ 2,239 görüntüden 28 öznitelik çıkarıldı
- ✅ features.csv oluşturuldu
- ✅ Görselleştirmeler kaydedildi

### 🎯 Sonraki Adımlar:

1. **Makine Öğrenmesi:** features.csv ile model eğitin (Random Forest, SVM, XGBoost)
2. **Deep Learning:** Sonuçları CNN modelleriyle karşılaştırın
3. **Hiperparametre Optimizasyonu:** Kernel boyutları, eşik değerleri
4. **Çoklu ROI Analizi:** 209 çoklu lezyon görüntüsünü inceleyin

### 📚 Kaynaklar:

- [ISIC Archive](https://challenge.isic-archive.com/)
- [GitHub Repository](https://github.com/KULLANICI_ADI/isic_segmentation_project)
- [Akademik Rapor](../docs/ISIC_2018_Segmentasyon_Raporu.docx)

---

**🙏 Teşekkürler!**

*Son Güncelleme: 10 Ocak 2026*